In [43]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [44]:
# Define the agent for the influenza simulation
class Person(Agent):
    def __init__(self, unique_id, model, status, resistensi, mobilitas, usia, vaccinated, symptom_severity):
        super().__init__(unique_id, model)
        self.status = status  # 'sehat', 'terinfeksi', 'sembuh'
        self.resistensi = resistensi
        self.mobilitas = mobilitas
        self.usia = usia
        self.vaccinated = vaccinated
        self.symptom_severity = symptom_severity
        self.days_infected = 0  # Track infection duration

    def step(self):
        if self.status == "terinfeksi":
            self.days_infected += 1
            # Check for recovery
            if np.random.random() < self.model.probabilitas_pemulihan:
                self.status = "sembuh"
            # Limit infection duration
            elif self.days_infected > self.model.infection_duration:
                self.status = "sembuh"
        elif self.status == "sehat":
            # Interact with neighbors and possibly get infected
            neighbors = self.model.grid.get_neighbors(self.pos, moore=True, radius=self.model.jangkauan_interaksi)
            for neighbor in neighbors:
                if neighbor.status == "terinfeksi" and np.random.random() < self.model.probabilitas_infeksi * (1 - self.resistensi):
                    self.status = "terinfeksi"
                    break

In [45]:
# Define the influenza model
class InfluenzaModel(Model):
    def __init__(self, num_agents, width, height, simulation_params, individual_data):
        super().__init__()
        self.num_agents = num_agents
        self.grid = MultiGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)

        # Simulation parameters
        self.probabilitas_infeksi = simulation_params["probabilitas_infeksi"].iloc[0]
        self.probabilitas_pemulihan = simulation_params["probabilitas_pemulihan"].iloc[0]
        self.jangkauan_interaksi = simulation_params["jangkauan_interaksi"].iloc[0]
        self.infection_duration = simulation_params["infection_duration"].iloc[0]

        # Create agents
        for _, row in individual_data.iterrows():
            agent = Person(row["id"], self, row["status"], row["resistensi"], row["mobilitas"],
                           row["usia"], row["vaccinated"], row["symptom_severity"])
            self.schedule.add(agent)
            x = self.random.randrange(width)
            y = self.random.randrange(height)
            self.grid.place_agent(agent, (x, y))

        # Data collector
        self.datacollector = DataCollector(
            {"Sehat": lambda m: self.count_status(m, "sehat"),
             "Terinfeksi": lambda m: self.count_status(m, "terinfeksi"),
             "Sembuh": lambda m: self.count_status(m, "sembuh")}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

    @staticmethod
    def count_status(model, status):
        return sum([1 for a in model.schedule.agents if a.status == status])

In [46]:
# Load the data
simulation_parameters = pd.read_csv('simulation_parameters.csv')
individual_data = pd.read_csv('individual_data.csv')

In [47]:
# Simulation parameters
width, height = 20, 20  # Grid size
simulation_steps = 100  # Number of steps in the simulation

In [48]:
# Run the model
influenza_model = InfluenzaModel(len(individual_data), width, height, simulation_parameters, individual_data)
for i in range(simulation_steps):
    influenza_model.step()

In [49]:
# Collect and visualize results
model_data = influenza_model.datacollector.get_model_vars_dataframe()

In [50]:
# Visualization 1: Line chart for all statuses
fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=model_data.index,
    y=model_data["Sehat"],
    mode='lines+markers',
    name='Sehat',
    line=dict(width=2, dash='solid'),
    marker=dict(size=6)
))
fig1.add_trace(go.Scatter(
    x=model_data.index,
    y=model_data["Terinfeksi"],
    mode='lines+markers',
    name='Terinfeksi',
    line=dict(width=2, dash='dash'),
    marker=dict(size=6)
))
fig1.add_trace(go.Scatter(
    x=model_data.index,
    y=model_data["Sembuh"],
    mode='lines+markers',
    name='Sembuh',
    line=dict(width=2, dash='dot'),
    marker=dict(size=6)
))
fig1.update_layout(
    title="Tren Status Kesehatan Selama Simulasi",
    xaxis_title="Waktu (langkah)",
    yaxis_title="Jumlah Individu",
    template="plotly_white"
)

fig1.show()

In [51]:
# Visualization 2: Pie chart at the final step
final_step = model_data.iloc[-1]
fig2 = px.pie(
    names=["Sehat", "Terinfeksi", "Sembuh"],
    values=[final_step["Sehat"], final_step["Terinfeksi"], final_step["Sembuh"]],
    title="Distribusi Status Kesehatan di Langkah Terakhir",
    template="plotly_white"
)
fig2.show()

In [52]:
# Visualization 3: Area chart for cumulative trends
fig3 = px.area(
    model_data,
    x=model_data.index,
    y=["Sehat", "Terinfeksi", "Sembuh"],
    title="Perubahan Status Kesehatan Secara Kumulatif",
    labels={"index": "Waktu (langkah)", "value": "Jumlah Individu", "variable": "Status Kesehatan"},
    template="plotly_white"
)
fig3.show()

In [53]:

# Save the results to a CSV file
model_data.to_csv("simulation_results.csv", index=False)